## Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

В колонках датасета содержатся следующие данные:  
* longitude — широта;
* latitude — долгота;
* housing_median_age — медианный возраст жителей жилого массива;
* total_rooms — общее количество комнат в домах жилого массива;
* total_bedrooms — общее количество спален в домах жилого массива;
* population — количество человек, которые проживают в жилом массиве;
* households — количество домовладений в жилом массиве;
* median_income — медианный доход жителей жилого массива;
* median_house_value — медианная стоимость дома в жилом массиве;
* ocean_proximity — близость к океану.

# Подготовка данных

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, RegressionEvaluator

In [33]:
spark = SparkSession.builder.master('local').appName('Colifornia Hosting').getOrCreate()

In [34]:
df_housing = spark.read.load('housing.csv', format='csv', sep=',',inferSchema=True, header=True)
df_housing.printSchema()
df_housing.show(5)

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR B

In [35]:
display(pd.DataFrame(df_housing.dtypes, columns=['columns', 'type']))

,columns,type
0,longitude,double
1,latitude,double
2,housing_median_age,double
3,total_rooms,double
4,total_bedrooms,double
5,population,double
6,households,double
7,median_income,double
8,median_house_value,double
9,ocean_proximity,string


In [36]:
df_housing.toPandas().describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


Разделим колонки на два типа: числовые и текстовые, которые представляют категориальные данные.

Заполним пропущенные значения в данных:

In [37]:
# проверим наличие пропущеных значений
df_housing.select([F.count(F.when(F.isnan(x) | F.col(x).isNull(), x)).alias(x) for x in df_housing.columns]).show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|        0|       0|                 0|          0|           207|         0|         0|            0|                 0|              0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+



In [38]:
subset = ['longitude', 'latitude', 'housing_median_age','total_rooms','total_bedrooms','population','households', 'median_income', 'median_house_value']
df_housing = df_housing.na.fill(0, subset)

In [39]:
df_housing.printSchema()

root
 |-- longitude: double (nullable = false)
 |-- latitude: double (nullable = false)
 |-- housing_median_age: double (nullable = false)
 |-- total_rooms: double (nullable = false)
 |-- total_bedrooms: double (nullable = false)
 |-- population: double (nullable = false)
 |-- households: double (nullable = false)
 |-- median_income: double (nullable = false)
 |-- median_house_value: double (nullable = false)
 |-- ocean_proximity: string (nullable = true)



Разделим колонки с признаками на числовые и категориальные, а также выделим целевой признак:

In [40]:
categorical_cols = ['ocean_proximity']
numerical_cols  = ['longitude', 'latitude', 'housing_median_age','total_rooms','total_bedrooms','population','households', 'median_income']
target = 'median_house_value' 

Трансформируем категориальные признаки с помощью трансформера StringIndexer:

In [41]:
indexer = StringIndexer(inputCols=categorical_cols, outputCols=[i + '_idx' for i in categorical_cols])
df_housing = indexer.fit(df_housing).transform(df_housing)
cols = [i for i in df_housing.columns for j in categorical_cols if (i.startswith(j))]
df_housing.select(cols).show(3)

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



Теперь преобразуем колонку с категориальными значениями с помощью OHE:

In [42]:
encoder = OneHotEncoder(inputCols=[i + '_idx' for i in categorical_cols], outputCols=[i + '_ohe' for i in categorical_cols])
df_housing = encoder.fit(df_housing).transform(df_housing)
cols = [i for i in df_housing.columns for j in categorical_cols if (i.startswith(j))]
df_housing.select(cols).show(3)

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



Далее следует объединение признаков в один вектоp:

In [43]:
categorical_assembler = VectorAssembler(inputCols=[i + '_ohe' for i in categorical_cols], outputCol='categorical_features')
df_housing = categorical_assembler.transform(df_housing)
df_housing.show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|categorical_features|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+-------------------+-------------------+--------------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|       (4,[3],[1.0])|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|                3.0|      (4,[3],[1.0])|       (4,[3],[1.0])|
|  -1

Следующим шагом проведём шкалирование значений, чтобы сильные выбросы не смещали предсказания модели:

In [44]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol="numerical_features")
df_housing = numerical_assembler.transform(df_housing)

In [45]:
standardScaler = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
df_housing = standardScaler.fit(df_housing).transform(df_housing)

In [46]:
print(df_housing.columns)

['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income', 'median_house_value', 'ocean_proximity', 'ocean_proximity_idx', 'ocean_proximity_ohe', 'categorical_features', 'numerical_features', 'numerical_features_scaled']


Финальный шаг - собрать трансформированные категорийные и числовые признаки с помощью VectorAssembler:

In [47]:
all_features = ['categorical_features','numerical_features_scaled']
final_assembler = VectorAssembler(inputCols=all_features, outputCol='features')
df_housing = final_assembler.transform(df_housing)
df_housing.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[3],[1.0])|     [-61.007269596069...|
|       (4,[3],[1.0])|     [-61.002278409814...|
|       (4,[3],[1.0])|     [-61.012260782324...|
+--------------------+-------------------------+
only showing top 3 rows



Разделяем датасет на обучающую и тестовую выборки (80 и 20 процентов соответсвенно):

In [48]:
train_data, test_data = df_housing.randomSplit([0.8, 0.2], seed=123)
print(train_data.count(), test_data.count())

16442 4198


# Обучение моделей

Модель с использованием всех данных файла   
Для построения модели используем оценщик LinearRegression из библиотеки MLlib:

In [49]:
lr = LinearRegression(labelCol=target, featuresCol='features', regParam=0.000000001)
model_full_lr = lr.fit(train_data)

In [50]:
prediction = model_full_lr.transform(test_data)
predictedLabel = prediction.select(target, 'prediction')

Найдем метрики RMSE, MAE и R2 для модели:

In [51]:
rmse_full = RegressionEvaluator(metricName="rmse", labelCol="median_house_value").evaluate(prediction)
mae_full = RegressionEvaluator(metricName="mae", labelCol="median_house_value", predictionCol="prediction").evaluate(prediction)
r2_full = RegressionEvaluator(metricName="r2", labelCol="median_house_value", predictionCol="prediction").evaluate(prediction)

Модель с использованием только числовых данных

In [52]:
lr_num = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled', regParam=0.000000001)

model_num = lr_num.fit(train_data) 

In [53]:
prediction = model_num.transform(test_data)

predictedLabes = prediction.select(target, 'prediction')
predictedLabes.show(5) 

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          103600.0|100347.21582292579|
|          106700.0| 191795.5737676164|
|           68400.0|  79420.5380367334|
|           90100.0|165894.42445842922|
|           72200.0|130645.09842462838|
+------------------+------------------+
only showing top 5 rows



In [54]:
rmse_num = RegressionEvaluator(metricName="rmse", labelCol="median_house_value").evaluate(prediction)
mae_num = RegressionEvaluator(metricName="mae", labelCol="median_house_value", predictionCol="prediction").evaluate(prediction)
r2_num = RegressionEvaluator(metricName="r2", labelCol="median_house_value", predictionCol="prediction").evaluate(prediction)

In [55]:
display(pd.DataFrame({'model':['RMSE', 'MAE', 'R2'],
                      'full_data': [rmse_full, mae_full, r2_full],
                      'num_data':[rmse_num, mae_num, r2_num]}))


,model,full_data,num_data
0,RMSE,67644.698765,68460.885279
1,MAE,49952.517909,50999.430109
2,R2,0.649457,0.640947


# Анализ результатов

По итогам исследования можно сделать вывод, что модель линейной регрессии наиболее точно предсказывает целевой признак на основе использования всех данных, как числовых так и категориальных, на что указывает меньшее значения метрик RMSE и MAE - , а так же большее значение метрики R2 - 0.649 против 0.640 у модели с использованием только числовых признаков.